In [390]:
import numpy as np
import graph
import digraph
from graph import Graph
from digraph import DiGraph
from collections import defaultdict
from itertools import tee, izip
import copy
import random
from random import shuffle
import time

## Preparation Functions/Classes

In [391]:
def read_instance_file(filename):
    '''
    ::param: filename: string, filename that indicates the location of instance data file
    ::return value: (specification, data)
    :: specification: dict, specification of the instance
    :: data: the numpy array with a list of edges and their cost, demand
    :: data: [vertex1 vertex2 cost demand]
    '''
    fd = open(inputfile)
    content = fd.readlines()
    content = [x.strip() for x in content] 
    specification = dict()
    for i in range(8):
        line = content[i].split(':')
        specification[line[0].strip()] = line[1].strip()
    # print(specification)
    data = list()
    for line in content[9:-1]:
        tmp = line.split()
        data.append([int(x.strip()) for x in tmp])
    data = np.array(data)
    fd.close()
    return specification, data

In [392]:
'''
filelist CARP_samples
egl-e1-A.dat  gdb10.dat  val1A.dat  val7A.dat
egl-s1-A.dat  gdb1.dat   val4A.dat
'''
inputfile = '../CARP_samples/egl-s1-A.dat'

## Read the graph into Data Structure

In [393]:
spec, data = read_instance_file(inputfile)
gf = Graph()
gf.load_from_data(data.tolist())

In [394]:
print(spec)

{'NON-REQUIRED EDGES': '115', 'VEHICLES': '7', 'CAPACITY': '210', 'NAME': 'egl-s1-A', 'TOTAL COST OF REQUIRED EDGES': '1394', 'REQUIRED EDGES': '75', 'VERTICES': '140', 'DEPOT': '1'}


In [395]:
capacity = int(spec['CAPACITY'])
depot = int(spec['DEPOT'])

## Solution-related

### output example

```
s 0,(1,2),(2,4),(4,1),0,0,(4,3),(3,1),0
q 15
```

In [396]:
def solution_verify(partitioned_solution, graph):
    loadlist = list()
    cost_total = 0
    costlist = list()
    for route in partitioned_solution:
        # costlist.append(route)
        load = 0
        prev_task = None
        cost_total += graph.get_shortest_path(route[0][0], depot)[1]
        # costlist.append(cost_total)
        for task in route:
            #print(task)
            if prev_task is not None:
                cost_total += graph.get_shortest_path(prev_task[1], task[0])[1]
                # costlist.append(cost_total)
            load += graph[task[0]][task[1]]['demand']
            cost_total += graph[task[0]][task[1]]['cost']
            # costlist.append(cost_total)
            prev_task = task
        loadlist.append(load)
        cost_total += graph.get_shortest_path(route[-1][1], depot)[1]
        costlist.append(cost_total)
    return loadlist, costlist,cost_total,sum(loadlist)

In [397]:
loadlist, costlist, cost_total, load_total = solution_verify(partitioned2, gf)
#print(load_total, cost_total)

In [400]:
output = 's 0,(1,116),(116,117),(117,2),(117,119),(118,114),(114,113),(113,112),(112,107),(107,108),(107,110),(110,111),(105,104),0,0,(112,110),(107,106),(106,105),(104,102),(66,67),(67,68),(67,69),(69,71),(71,72),(72,73),(73,44),(44,45),(45,34),0,0,(87,86),(86,85),(85,84),(84,82),(82,80),(80,79),(79,78),(78,77),(77,46),(46,43),(43,37),(37,36),(36,38),(38,39),(39,40),0,0,(108,109),0,0,(66,62),(62,63),(63,64),(64,65),(56,55),(55,54),(55,140),(49,48),0,0,(124,126),(126,130),(43,44),(34,139),(139,33),(33,11),(11,12),(12,13),(20,22),0,0,(95,96),(96,97),(97,98),(140,49),(11,27),(27,28),(28,30),(30,32),(28,29),(27,25),(25,24),(24,20),0,0,(11,8),(8,6),(6,5),(8,9),(13,14),0\nq 6201'

In [404]:
solution =string_extract(output)
solution_verify(solution[0],gf)

([210, 210, 208, 103, 106, 210, 208, 139],
 [498, 1142, 1784, 2226, 2907, 3986, 5183, 6201],
 6201,
 1394)

In [398]:
import re
class InputError(Exception):
    def __init__(self, message):
        self.msg = message

def string_extract(result_string):
    splitted = result_string.splitlines()
    solution = None
    solution = list()
    result_cost = 0
    try:
        for line in splitted:
            if len(line)<3:
                raise InputError('Length not right')
            if line[0] == 's':
                line = line[2:]
                pattern_route = r'(\([\d]+,[\d]+\),?)+'
                pattern_task = r'\(([\d]+),([\d]+)\)'
                for m in re.finditer(pattern_route, line):
                    route = list()
                    for m_sub in re.finditer(pattern_task, m.group(0)):
                        route.append((int(m_sub.group(1)), int(m_sub.group(2))))
                    solution.append(route)
            elif line[0] == 'q':
                quality = line.split(' ')
                try:
                    result_cost = int(quality[1])
                    # print(result_cost)
                except ValueError:
                    #Handle the exception
                    raise InputError('Please enter an integer')
    except InputError as err:
        print(err.msg)
    return solution, result_cost